# MiniGo

## 使用 Swift 软件包管理器安装 MiniGo

In [ ]:
%install-swiftpm-flags -c release -Xlinker -ltensorflow
%install '.package(url: "https://github.com/tensorflow/swift-models", .branch("tf-roadshow-19"))' MiniGo

## 导入现有模型

In [ ]:
import TensorFlow
import MiniGo
import Foundation

In [ ]:
system("""
    mkdir -p MiniGo
    gsutil cp 'gs://minigo-pub/v15-19x19/models/000939-heron.*' MiniGo/
    """)

In [ ]:
let boardSize = 19
let modelConfig = ModelConfiguration(boardSize: boardSize)
var model = GoModel(configuration: modelConfig)

let reader = PythonCheckpointReader(path: "MiniGo/000939-heron")
model.load(from: reader)

## 设置游戏

In [ ]:
let simulationCountForOneMove = 40

let config = GameConfiguration(
    size: boardSize,
    komi: 7.5,
    isVerboseDebuggingEnabled: false
)

let predictor = MCTSModelBasedPredictor(boardSize: boardSize, model: model)

let mctsConfiguration = MCTSConfiguration(
    gameConfiguration: config,
    simulationCountForOneMove: simulationCountForOneMove
)

## 玩一轮

In [ ]:
try playOneGame(
    gameConfiguration: config,
    participants: [
        MCTSPolicy(participantName: "black", predictor: predictor,
                   configuration: mctsConfiguration),
        MCTSPolicy(participantName: "white", predictor: predictor,
                   configuration: mctsConfiguration),
    ])